In [1]:
import solana
from solana.publickey import PublicKey
from solana.rpc.api import Client
from solana.keypair import Keypair
from solana import system_program
from solana import transaction
from solana.system_program import SYS_PROGRAM_ID, CreateAccountParams, create_account
from solana.transaction import *
from spl.token.constants import TOKEN_PROGRAM_ID, ASSOCIATED_TOKEN_PROGRAM_ID
from spl.token.instructions import initialize_mint, InitializeMintParams, get_associated_token_address
from spl.token._layouts import MINT_LAYOUT
from spl.token.client import Token
from spl.token import instructions
import base64
from borsh_construct import *
from solana.rpc.commitment import Commitment
import time
import requests
import concurrent.futures


client = Client("https://api.devnet.solana.com")

client.is_connected()

True

In [343]:
def create_num_cnt(num):
    return [int((num-num%255)/255),255,num%255]
proposal = "Helly world, This is my first proposal ever. Do you think it will work?"
# proposal = proposal.encode(encoding = 'UTF-8', errors = 'strict')
choices = ["No, Not at all", "Yes, Ofccourse", "Yes, Eventually", "Clueless", "hello world"]


proposal += '\0' * (500-len(proposal));

choices = [choice + '\0'* (30-len(choice)) for choice in choices]
# print ("length of {0} is {1}".format(repr(mystring), len(mystring)))



# len(proposal)

In [344]:
program_id = PublicKey("DrSryh4M2nYyRSqgDNMiASK73LSoZrN56oHWXaAcnmCt")
sys_program_meta = AccountMeta(SYS_PROGRAM_ID, False, False)
spl_program_meta  = AccountMeta(TOKEN_PROGRAM_ID, False, False)

In [345]:
def getAccountInfo(client: Client, account_pk: PublicKey):
    """Fetch account information from RPC, parse out the data and deserialize"""
    comm = Commitment("confirmed")
    res = client.get_account_info(account_pk, comm, encoding='base64')
    data = res['result']
    if isinstance(data, dict):
        return accountSchema.parse(base64.urlsafe_b64decode(data['value']['data'][0]))
    else:
        raise AttributeError(f'Unknown RPC result {data}')

In [346]:
seed = bytes([125,215,212,54,213,98,45,123]*4)
payer_keypair = Keypair.from_seed(seed)


In [347]:
accountSchema = CStruct(
    "proposal_id" / U32
)

(proposals_pda, proposals_pda_bump) = PublicKey.find_program_address([b"Dsol_Dao_Governance"], program_id)

proposals = getAccountInfo(client,proposals_pda)
proposals

Container(proposal_id=0)

In [348]:
proposal_pda, _ = PublicKey.find_program_address([b"Dsol_Dao_Governance", proposals.proposal_id.to_bytes(4,"little") ], program_id)
metadata_program_pubkey = PublicKey("metaqbxxUerdq28cj1RbAWkYQm3ybzjb6a8bt518x1s")
governor_account_meta = AccountMeta(PublicKey("GFUQwvXZwHVhnoeyDf1i621j3SaZWY4zXN4Z9Kfpw66M"), False, False)
(metadata_pda, metadata_nonce) = PublicKey.find_program_address([b'metadata', bytes(metadata_program_pubkey), bytes(governor_account_meta.pubkey)], metadata_program_pubkey)



payer_account_meta = AccountMeta(payer_keypair.public_key, True, True)
proposals_account_meta = AccountMeta(proposals_pda, False, True)
proposal_pda_meta = AccountMeta(proposal_pda, False, True)
sysvar_clock_meta = AccountMeta(solana.sysvar.SYSVAR_CLOCK_PUBKEY, False, False)
metadata_account_meta = AccountMeta(metadata_pda, False, True)
governor_associated_meta = AccountMeta(get_associated_token_address(payer_keypair.public_key, governor_account_meta.pubkey), False, False)



In [349]:
accounts = [
    payer_account_meta,
    governor_account_meta,
    proposals_account_meta,
    proposal_pda_meta,
    sysvar_clock_meta,
    metadata_account_meta,
    governor_associated_meta,

    sys_program_meta
]

proposal_pda_meta


AccountMeta(pubkey=3KV9CU1asMRnbBhSMHnLxGfDGHoKXrmvGY9go87utWCH, is_signer=False, is_writable=True)

In [350]:
instruction_data = bytes([1])

instruction_data += bytes(proposal, "ascii")
for choice in choices:
    instruction_data += bytes(choice, "ascii")

instruction_data += bytes(create_num_cnt(65279))
# instruction_data

In [351]:
transaction = Transaction()
transaction.add(TransactionInstruction(
    accounts,
    program_id,
    instruction_data
))
f"https://explorer.solana.com/tx/{client.send_transaction(transaction, payer_keypair)['result']}?cluster=devnet"

RPCException: {'code': -32002, 'message': 'Transaction simulation failed: Error processing Instruction 0: custom program error: 0x0', 'data': {'accounts': None, 'err': {'InstructionError': [0, {'Custom': 0}]}, 'logs': ['Program DrSryh4M2nYyRSqgDNMiASK73LSoZrN56oHWXaAcnmCt invoke [1]', 'Program log: \nDATA_0: 1\n', 'Program log: \nPROPOSAL_PDA: 3KV9CU1asMRnbBhSMHnLxGfDGHoKXrmvGY9go87utWCH:             PROPOSAL_ID: [0, 0, 0, 0]\n', 'Program 11111111111111111111111111111111 invoke [2]', 'Create Account: account Address { address: 3KV9CU1asMRnbBhSMHnLxGfDGHoKXrmvGY9go87utWCH, base: None } already in use', 'Program 11111111111111111111111111111111 failed: custom program error: 0x0', 'Program DrSryh4M2nYyRSqgDNMiASK73LSoZrN56oHWXaAcnmCt consumed 31468 of 200000 compute units', 'Program DrSryh4M2nYyRSqgDNMiASK73LSoZrN56oHWXaAcnmCt failed: custom program error: 0x0']}}

In [ ]:
ProposalResult = CStruct(
    "leading_choice_bump" / U8,
    "choices" / U32[5],
    
)


Vote = CStruct(
    "proposal_pda" / U8[32],
   "mint_id" / U8[32],
    "choice_bumps" / U8[5]
)

accountSchema = CStruct(
    "proposal_pda" / U8[32],
    "proposal" / String,
    "choices" / String,
    "max_vote_bump" / U32,
    "proposal_governor_id" / U8[32],
    "duration" / U64,
    "ending_data" / U64,
    "votes" / Vec(Vote),
    "result" / ProposalResult

)

In [ ]:
Proposal = getAccountInfo(client, proposal_pda)

In [ ]:
choices = [Proposal.choices[i*30 : i*30 + 30].strip('\0') for i in range(5)]
proposal = Proposal.proposal.strip("\0")
[Proposal.max_vote_bump, proposal, Proposal.votes, choices, Proposal.result]

[4,
 'Helly world, This is my first proposal ever. Do you think it will work?',
 ListContainer([Container(proposal_pda=ListContainer([34, 115, 165, 222, 188, 143, 146, 123, 2, 89, 187, 125, 114, 65, 93, 180, 149, 32, 92, 22, 152, 186, 63, 109, 81, 52, 60, 2, 137, 189, 8, 90]), mint_id=ListContainer([56, 36, 229, 138, 101, 183, 91, 5, 204, 219, 103, 8, 81, 228, 151, 166, 171, 75, 92, 7, 132, 28, 85, 117, 109, 54, 36, 118, 132, 248, 242, 94]), choice_bumps=ListContainer([2, 1, 4, 3, 0])), Container(proposal_pda=ListContainer([34, 115, 165, 222, 188, 143, 146, 123, 2, 89, 187, 125, 114, 65, 93, 180, 149, 32, 92, 22, 152, 186, 63, 109, 81, 52, 60, 2, 137, 189, 8, 90]), mint_id=ListContainer([226, 148, 110, 90, 32, 179, 1, 238, 7, 64, 76, 236, 2, 253, 131, 243, 141, 70, 158, 87, 135, 201, 86, 237, 30, 32, 169, 129, 113, 99, 41, 138]), choice_bumps=ListContainer([2, 1, 4, 3, 0])), Container(proposal_pda=ListContainer([34, 115, 165, 222, 188, 143, 146, 123, 2, 89, 187, 125, 114, 65, 93, 180, 

In [ ]:
vote_pda, _ = PublicKey.find_program_address([b"Dsol_Dao_Governance", bytes(proposal_pda_meta.pubkey), Proposal.max_vote_bump.to_bytes(4, "big")], program_id)

governor_account_meta = AccountMeta(PublicKey("HzugB5CiTrSr3SZkN3p9cudYoA1R1swzgE68vpFwfQoP"), False, False)
governor_pda, _ = PublicKey.find_program_address([b"Dsol_Dao_Governance", bytes(proposal_pda_meta.pubkey), bytes(governor_account_meta.pubkey)], program_id)


metadata_program_pubkey = PublicKey("metaqbxxUerdq28cj1RbAWkYQm3ybzjb6a8bt518x1s")
(metadata_pda, metadata_nonce) = PublicKey.find_program_address([b'metadata', bytes(metadata_program_pubkey), bytes(governor_account_meta.pubkey)], metadata_program_pubkey)


governor_associated_meta = AccountMeta(get_associated_token_address(payer_keypair.public_key, governor_account_meta.pubkey), False, False)

accounts = [
    payer_account_meta,
    governor_account_meta,
    # AccountMeta(vote_pda, False, True),
    proposal_pda_meta,
    AccountMeta(governor_pda, False, True),
    AccountMeta(metadata_pda, False, False),
    governor_associated_meta,

    sys_program_meta,
    sys_program_meta
]

In [ ]:
transaction = Transaction()
transaction.add(TransactionInstruction(
    accounts,
    program_id,
    bytes([2,2,110,240,13,200] + create_num_cnt(Proposal.max_vote_bump))
))
f"https://explorer.solana.com/tx/{client.send_transaction(transaction, payer_keypair)['result']}?cluster=devnet"

'https://explorer.solana.com/tx/4MfbKTFwv2ptSLEBWRf8GfupD8msprqGW9AdK7LB2KjdkoHth9svRxZwQzCNv1h8d8DaniGRqafDzLoKndjVnWQs?cluster=devnet'

In [ ]:
accounts = [proposal_pda_meta]

transaction = Transaction()
transaction.add(TransactionInstruction(
    accounts,
    program_id,
    bytes([4] + create_num_cnt(Proposal.max_vote_bump))
))
f"https://explorer.solana.com/tx/{client.send_transaction(transaction, payer_keypair)['result']}?cluster=devnet"

'https://explorer.solana.com/tx/3fwmWA6Wv3zEvNvr2YpJVH1DQpcQyzk5GGWBEBRe49p4vJVUoD2n9jARFJKJd5esEo7jsjXRCAhamUNSEfSdL1Aj?cluster=devnet'